In [366]:
import pandas as pd
df = pd.read_feather('../data/chatgpt/chatgpt_yaml.feather')

In [367]:
print("Number of playbooks created:", (df.output != '').sum())

Number of playbooks created: 130


In [368]:
df = df[df['valid'] == 1]

In [369]:
df = df[:100]

In [370]:
df.reset_index(drop=True, inplace=True)

In [350]:
df

,title,body,prompt,output,valid
0,"""is defined"" reporting false for variable that...",### Summary\n\nGiven the following variable:\r...,Create an Ansible playbook to exhibit the foll...,\n\n---\n\n- hosts: all\n become: true\n tas...,1
1,Systemd user services cannot be enabled globally,### Summary\n\nWhen trying to install a system...,Create an Ansible playbook to exhibit the foll...,\n\n---\n- name: Install yum on RHEL5\n hosts...,1
2,Conditionnal bug report,"### Summary\n\nHello, I am just trying to chec...",Create an Ansible playbook to exhibit the foll...,\n\n---\n- name: install yum package on RHEL5\...,1
3,apt_repository: The repository ... does not h...,### Summary\r\n\r\nGiven: \r\n\r\n```\r\n- na...,Create an Ansible playbook to exhibit the foll...,\n\n---\n- name: Install yum package on RHEL5\...,1
4,"gcp_compute_instance_facts problem with the ""~...","### Summary\n\nWhen I try to use the ""contains...",Create an Ansible playbook to exhibit the foll...,\n\n---\n- name: Install yum package on RHEL5\...,1
...,...,...,...,...,...
95,Gathering facts fails after setting login shel...,### Summary\n\nAfter setting login shell to [n...,Create an Ansible playbook to exhibit the foll...,"(""Please enter a number:"");\n\nif (num % 2 ===...",1
96,include_role with when condition causes exception,### Summary\n\nWhen including a role with \r\n...,Create an Ansible playbook to exhibit the foll...,\n\nA prompt is a message that appears on the ...,1
97,meta flush_handlers doesn't work in role,### Summary\n\nTriggering handlers with `ansib...,Create an Ansible playbook to exhibit the foll...,\n\nA prompt is a message or question that app...,1
98,ansible_fqdn has disappeared from ansible_facts,### Summary\r\n\r\nHaving come back to a playb...,Create an Ansible playbook to exhibit the foll...,(\n 'What is the name of the current Ja...,1


In [10]:
import torch
import numpy as np
from bert_score import score
scores = []
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
P, R, F1 = score(predictions, references, model_type="distilbert-base-uncased", device=device)
scores.append(F1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [72]:
df['output']

0     \n\n---\n\n- hosts: all\n  become: true\n  tas...
1     \n\n---\n- name: Install yum on RHEL5\n  hosts...
2     \n\n---\n- name: install yum package on RHEL5\...
3     \n\n---\n- name: Install yum package on RHEL5\...
4     \n\n---\n- name: Install yum package on RHEL5\...
                            ...                        
95    ("Please enter a number:");\n\nif (num % 2 ===...
96    \n\nA prompt is a message that appears on the ...
97    \n\nA prompt is a message or question that app...
98    (\n        'What is the name of the current Ja...
99    (message, defaultValue)\n\nconst message = 'Wh...
Name: output, Length: 100, dtype: object

In [361]:
import transformers
transformers.logging.set_verbosity_error()
import torch
import numpy as np
from bert_score import score
from tqdm import tqdm
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
scores = []
sentences = df['output']
i = 0
for j in range(i+1, len(sentences)):
    reference = [sentences[i]]
    candidate = [sentences[j]]
    P, R, F1 = score(candidate, reference, model_type="distilbert-base-uncased", device=device)
    scores.append((i, j, F1[0]))

print(scores)



KeyboardInterrupt: 

In [371]:
df['bert_score'] = ''

In [372]:
import transformers
transformers.logging.set_verbosity_error()
import torch
import numpy as np
from bert_score import score
from tqdm import tqdm

with tqdm(total=None, desc="Getting BertScore") as pbar:
    for i in df.index:
        scores = []
        if df.loc[i, 'bert_score'] == '':
            for j in df.index:
                if i != j:
                    reference = [df.loc[i, 'output']]
                    candidate = [df.loc[j, 'output']]
                    P, R, F1 = score(candidate, reference, model_type="distilbert-base-uncased", device=device)
                    scores.append(F1[0])
                else:
                    scores.append(torch.tensor(0))
        df.at[i, 'bert_score'] = scores
        pbar.update(1)

Getting BertScore: 100it [1:32:49, 55.69s/it]


In [373]:
df.to_pickle('bert.pickle')

In [73]:
import transformers
transformers.logging.set_verbosity_error()
import torch
import numpy as np
from bert_score import score
scores = []
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
predictions = [df['output'][0]]
references = [df['output'][2]]
P, R, F1 = score(predictions, references, model_type="distilbert-base-uncased", device=device)
scores.append(F1[0])

In [82]:
F1[0]

tensor(0.8792)